In [559]:
import numpy as np
import pandas as pd
import json
import re
import nltk
nltk.download('stopwords') #make sure list up to date
from nltk.corpus import wordnet
from IPython.display import clear_output


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\magar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [560]:
#read in scraped data
chartsDf = pd.read_csv('charts_and_lyrics_2017.csv', encoding='utf-8')

chartsDf

,chart,chartURL,rank,song,artist,lyrics
0,Overall,https://www.billboard.com/charts/year-end//201...,1,Shape Of You,Ed Sheeran,[Verse 1]\nThe club isn't the best place to fi...
1,Overall,https://www.billboard.com/charts/year-end//201...,2,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bieber,"[Intro: Luis Fonsi & Daddy Yankee]\nAy, ¡Fonsi..."
2,Overall,https://www.billboard.com/charts/year-end//201...,3,That's What I Like,Bruno Mars,"[Verse 1]\nHey, hey, hey\nI got a condo in Man..."
3,Overall,https://www.billboard.com/charts/year-end//201...,4,Humble.,Kendrick Lamar,[Intro]\nNobody pray for me\nIt's been that da...
4,Overall,https://www.billboard.com/charts/year-end//201...,5,Something Just Like This,The Chainsmokers & Coldplay,[Verse 1: Chris Martin]\nI've been reading boo...
5,Overall,https://www.billboard.com/charts/year-end//201...,6,Bad And Boujee,Migos Featuring Lil Uzi Vert,"[Intro: Offset]\nYou know, young rich niggas\n..."
6,Overall,https://www.billboard.com/charts/year-end//201...,7,Closer,The Chainsmokers Featuring Halsey,"[Verse 1: Andrew Taggart]\nHey, I was doing ju..."
7,Overall,https://www.billboard.com/charts/year-end//201...,8,Body Like A Back Road,Sam Hunt,"[Verse 1]\nGot a girl from the south side, got..."
8,Overall,https://www.billboard.com/charts/year-end//201...,9,Believer,Imagine Dragons,[Verse 1]\nFirst things first\nI'ma say all th...
9,Overall,https://www.billboard.com/charts/year-end//201...,10,Congratulations,Post Malone Featuring Quavo,"[Intro: Post Malone]\nMm-mmm\nYah, yah\nMm-mmm..."


In [561]:
def lineSplitAndClean(lyrics):    
    #make sure lyrics were found for this song
    #lyrics will be np.nan if none were found
    if not pd.isnull(lyrics):
        #strip "[Chorus]", "[Verse 1]" etc tags
        lyrics = re.sub(r'\[.*\]', '', lyrics)
        
        #splitting on \n instead of using sent_tokenize() from nltk as I don't
        #expect song lyrics to have great sentence structure so think \n will
        #be a more reliable delimiter
        #using .lower both for uniformity and cause stopwords list is all lowercase
        lines = [line.lower() for line in lyrics.split('\n') if len(line)>0]
        
        #strip any punctuation
        lines = [re.sub(r'\W',' ',line) for line in lines]

        #replace any multiple consecutive spaces with just one space
        lines = [re.sub(r'\s+',' ',line) for line in lines]
        
        #possible for lines to be an empty list here like for instrumental
        #songs the starting lyrics will just be '[instrumental]'
        if lines:
            return lines
        else:
            return np.nan
    else:
        return np.nan

In [562]:
#breaking lyrics into lines so nltk.pos_tag get's sentences passed to it rather 
#than all the words at once. Should give better context for tagging.
chartsDf['lines'] = chartsDf.lyrics.apply(lineSplitAndClean)

chartsDf

,chart,chartURL,rank,song,artist,lyrics,lines
0,Overall,https://www.billboard.com/charts/year-end//201...,1,Shape Of You,Ed Sheeran,[Verse 1]\nThe club isn't the best place to fi...,[the club isn t the best place to find a lover...
1,Overall,https://www.billboard.com/charts/year-end//201...,2,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bieber,"[Intro: Luis Fonsi & Daddy Yankee]\nAy, ¡Fonsi...","[ay fonsi d y , ohhh oh no oh no oh, hey yeah ..."
2,Overall,https://www.billboard.com/charts/year-end//201...,3,That's What I Like,Bruno Mars,"[Verse 1]\nHey, hey, hey\nI got a condo in Man...","[hey hey hey, i got a condo in manhattan, baby..."
3,Overall,https://www.billboard.com/charts/year-end//201...,4,Humble.,Kendrick Lamar,[Intro]\nNobody pray for me\nIt's been that da...,"[nobody pray for me, it s been that day for me..."
4,Overall,https://www.billboard.com/charts/year-end//201...,5,Something Just Like This,The Chainsmokers & Coldplay,[Verse 1: Chris Martin]\nI've been reading boo...,"[i ve been reading books of old, the legends a..."
5,Overall,https://www.billboard.com/charts/year-end//201...,6,Bad And Boujee,Migos Featuring Lil Uzi Vert,"[Intro: Offset]\nYou know, young rich niggas\n...","[you know young rich niggas, you know so we ai..."
6,Overall,https://www.billboard.com/charts/year-end//201...,7,Closer,The Chainsmokers Featuring Halsey,"[Verse 1: Andrew Taggart]\nHey, I was doing ju...","[hey i was doing just fine before i met you, i..."
7,Overall,https://www.billboard.com/charts/year-end//201...,8,Body Like A Back Road,Sam Hunt,"[Verse 1]\nGot a girl from the south side, got...",[got a girl from the south side got braids in ...
8,Overall,https://www.billboard.com/charts/year-end//201...,9,Believer,Imagine Dragons,[Verse 1]\nFirst things first\nI'ma say all th...,"[first things first, i ma say all the words in..."
9,Overall,https://www.billboard.com/charts/year-end//201...,10,Congratulations,Post Malone Featuring Quavo,"[Intro: Post Malone]\nMm-mmm\nYah, yah\nMm-mmm...","[mm mmm, yah yah, mm mmm, yah hey , my momma c..."


In [563]:
def convertPOSTag(treebank_tag):
    #nltk.pos_tag() uses tags from the treebank corpus 
    #https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    #but the lemmatiser uses tags from word net so need to convert.
    #only noun, verb, adjective and adverb are accepted.
    #adapted from https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
    if treebank_tag.startswith('J'):
        return wordnet.ADJ #returns 'a'
    elif treebank_tag.startswith('V'):
        return wordnet.VERB #returns 'v'
    elif treebank_tag.startswith('R'):
        return wordnet.ADV #returns 'r'

    #return noun for both actual nouns and anything that doesn't fit the other three 
    #noun is the default for the lemmatiser anyway so this will have same effect 
    #as not passing a tag.
    else:
        return wordnet.NOUN #returns 'n'

In [564]:
#needed corrections found by looking at top 25 of noun, verb, adj and adverb dfs

#high ranking slang words I noticed that aren't getting their correct lemmas
lemmaCorrections = {'wan':'want', #from wanna
                    'gon':'go', #from gonna, gon.
                    'lil':'little',
                    'hol':'hold' #from "hol' up" from LOVE. by Kendrick Lamar
                   }

stopwords = nltk.corpus.stopwords.words('english') #are lowercase
nltkStopwordsWithAdditions = stopwords + \
                             ['la', #comes up a lot (like 'la-la-la') but isn't a word so don't include it
                              'eh', 'ayy', #same as 'la'. 'eh' as in pronounced like 'aye'
                              'oh', 'ooh', #an exclamation with no real meaning but ranks highly so exclude
                              'us', 'u', 'ya', #different forms of 'you' which is a stopword
                              'na', 'ta', #alt forms of 'to' which is a stopword. From 'gonna', 'wanna', 'gotta'
                              'nah', #usage a mix of a sung-only word and slang for 'no' which is a stopword
                              'po', #coming from the 
                              'doo', 'ah', 'uh' #sung-only words
                             ]

In [565]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def countLemmas(row):    
    lines = row.lines
    chartName = row.chart
    
    #make sure lyrics were found for this song
    #lyrics will be np.nan if none were found
    try:
        if pd.isnull(lines): #causes an ValueError when lines is an array (eg [[x], [y]])
            return
    except ValueError:
        pass
    
    for line in lines:
        words = nltk.word_tokenize(line)

        #gives list of tuples in form [(car, NN), (run, VB)... etc]
        word_tags = nltk.pos_tag(words)

        #add lemmas and words to count list
        for wordAndTag in word_tags:
            word = wordAndTag[0]
            
            tag = convertPOSTag(wordAndTag[1])
            
            if word not in nltkStopwordsWithAdditions: 
                lemma = lemmatizer.lemmatize(word, tag)
                
                #correct known misclassified lemmas
                if lemma in lemmaCorrections.keys():
                    lemma = lemmaCorrections[lemma]
                
                lemmaAndTag = (lemma, tag)
                
                #for getting info on words that look like they've been misclassiffied
                #if lemma == 'going':
                #    print(word, '--', lemma, '\n', row.song, '--', row.artist, '\n', line)
                #break

                #add entry to dict with starting values
                if lemmaAndTag not in lemmaCounts.keys():
                    lemmaCounts[lemmaAndTag] = {'allChartsCount': 0, 
                                                'sourceWords': []}
                    #add counters for each chart (eg overall, rock, etc)
                    for chart in chartsDf.chart.unique():
                        lemmaCounts[lemmaAndTag][chart] = 0

                lemmaCounts[lemmaAndTag]['allChartsCount'] = \
                lemmaCounts[lemmaAndTag]['allChartsCount'] + 1

                lemmaCounts[lemmaAndTag][chartName] = \
                lemmaCounts[lemmaAndTag][chartName] + 1

                #keep a list of words that are feeding into this lemma for sanity checking
                if word not in lemmaCounts[lemmaAndTag]['sourceWords']:
                    lemmaCounts[lemmaAndTag]['sourceWords'].append(word)

    return

In [566]:
lemmaCounts = {}
chartsDf.apply(countLemmas, axis=1)

#get rid of 600 Nones printed out
clear_output()

In [567]:
lemmaCountsDf = pd.DataFrame.from_dict(lemmaCounts, orient='index')

#currently the index of the df is the words and the counts are in a column called 0
lemmaCountsDf.sort_values(by=['allChartsCount'], ascending=False, inplace=True)

#create an actual index and move the lemmas and their tags into columns
lemmaCountsDf.reset_index(inplace=True)

#change the index from starting at 0 to start at 1 so it's easier to see the rank of the rows
lemmaCountsDf.index = lemmaCountsDf.index + 1

lemmaCountsDf.rename(columns={'level_0':'lemma', 'level_1':'wordType'}, inplace=True)

lemmaCountsDf

,lemma,wordType,allChartsCount,sourceWords,Overall,Rock,Country,R&B/Hip-Hop,Dance/Electronic,Christian
1,get,v,2432,"[get, got, getting, gets, gotten]",563,275,335,792,311,156
2,know,v,1743,"[know, knew, known, knows, knowing]",389,181,204,454,321,194
3,like,n,1647,"[like, likes]",381,137,263,465,226,175
4,yeah,n,1547,[yeah],400,64,184,644,178,77
5,go,v,1426,"[go, going, went, gone, gon, goes]",235,154,250,244,324,219
6,want,v,1246,"[want, wan, wanted, wants, wanting]",267,124,186,324,237,108
7,make,v,942,"[make, made, making, makes]",171,110,196,207,73,185
8,say,v,913,"[say, said, says, saying]",207,96,118,218,183,91
9,come,v,909,"[come, came, coming, comes]",236,141,85,169,118,160
10,take,v,843,"[take, took, taking, takes, taken]",149,220,109,152,104,109


In [568]:
def topLemmasForAWordTypeByChart(wordType):
    concatenatedDf = lemmaCountsDf[lemmaCountsDf.wordType == wordType] \
                     .sort_values(by='allChartsCount', 
                                  ascending=False, 
                                  inplace=False)[['lemma']]
    concatenatedDf.rename(columns={'lemma':'allCharts'}, inplace=True)
    concatenatedDf.reset_index(inplace=True, drop=True)

        
    for chart in chartsDf.chart.unique():
        tempDf = lemmaCountsDf[lemmaCountsDf.wordType == wordType] \
                 .sort_values(by=[chart], 
                              ascending=False, 
                              inplace=False)[['lemma']]

        tempDf.rename(columns={'lemma':chart}, inplace=True)
        tempDf.reset_index(inplace=True, drop=True)

        concatenatedDf = pd.concat([concatenatedDf, tempDf], axis=1)
    
    #reset index to start from 1
    concatenatedDf.index = concatenatedDf.index + 1
    
    return concatenatedDf


In [569]:
topNounsDf = topLemmasForAWordTypeByChart('n')
topNounsDf.head(25)

,allCharts,Overall,Rock,Country,R&B/Hip-Hop,Dance/Electronic,Christian
1,like,yeah,time,like,yeah,like,love
2,yeah,like,like,yeah,like,yeah,like
3,love,baby,love,girl,bitch,love,god
4,one,bitch,cause,time,nigga,way,heart
5,time,one,one,baby,girl,one,one
6,way,time,heart,one,one,cause,life
7,baby,love,life,every,baby,dance,name
8,bitch,way,way,way,man,heart,every
9,girl,nigga,everything,night,fuck,baby,time
10,cause,girl,yeah,world,time,time,way


In [570]:
topVerbsDf = topLemmasForAWordTypeByChart('v')
topVerbsDf.head(25)

,allCharts,Overall,Rock,Country,R&B/Hip-Hop,Dance/Electronic,Christian
1,get,get,get,get,get,go,go
2,know,know,take,go,know,know,know
3,go,want,know,know,want,get,see
4,want,come,go,make,go,want,make
5,make,go,come,want,say,say,come
6,say,say,feel,say,make,let,get
7,come,make,want,take,need,feel,let
8,take,feel,make,come,come,love,take
9,let,take,say,think,take,come,want
10,feel,love,run,see,fuck,take,love


In [571]:
topAdjectivesDf = topLemmasForAWordTypeByChart('a')
topAdjectivesDf.head(25)

,allCharts,Overall,Rock,Country,R&B/Hip-Hop,Dance/Electronic,Christian
1,good,low,good,little,young,good,good
2,little,good,hard,good,good,low,open
3,young,young,high,last,real,high,rich
4,low,bad,last,old,little,little,whole
5,high,little,free,long,much,side,hard
6,real,real,black,high,nigga,bad,alive
7,bad,old,easy,black,bad,okay,strong
8,old,cold,much,whole,new,young,old
9,last,wild,little,red,big,long,right
10,much,new,young,crazy,wild,mum,new


In [572]:
topAdverbsDf = topLemmasForAWordTypeByChart('r')
topAdverbsDf.head(25)

,allCharts,Overall,Rock,Country,R&B/Hip-Hop,Dance/Electronic,Christian
1,never,back,back,never,back,never,never
2,back,never,never,back,never,back,back
3,right,ever,still,right,even,ever,away
4,ever,away,away,ever,still,well,right
5,still,well,right,always,ever,alone,still
6,even,right,even,well,right,right,alone
7,away,yeah,alone,still,yeah,even,ever
8,well,even,ahead,even,enough,away,even
9,alone,still,enough,away,well,still,always
10,always,bust,ever,little,really,lonely,together
